# Intro to Prompt Engineering with LLMs

[This course](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) is produced by the ubiquitous Andrew Ng & OpenAI Engineer Isa Fulford.

The course starts with a distinction in the functionality between 2 types of LLM:

**Base LLM**: Goos at predicting the next predicted word, so can exhibit autocomplete
behaviour. 

**Instruction Tuned LLM**: A base LLM trained to respond to instructions such as 
questions. This often involves **RLHF** - reinforcement learning with human feedback. 

The LLM output should be:

**H**elpful, **H**onest & **H**armless

## Prompt Engineering

To follow along, you'll need:
* to pip install `openai` - this is in the requirements.txt.
* to generate an api key from the [openai website](https://platform.openai.com/account/api-keys).


In [20]:
import toml
import os
from pyprojroot import here
import openai

In [26]:
CONFIG = toml.load(os.path.join(here(), ".ignore_me", "secrets.toml"))
PROMPT_ENGINEER = CONFIG["openai"]["PROMPT_ENGINEER"]
openai.api_key = PROMPT_ENGINEER


In [22]:
def get_answer(usr_input, mod="gpt-3.5-turbo"):
    """Get an answer from chat GPT.

    Args:
        usr_input (str): A prompt generated by the user.
        mod (str, optional): The model of GPT to use. Defaults to "gpt-3.5-turbo".

    Returns:
        str: The model response text.
    """
    mod_input = [{"role": "user", "content": usr_input}]
    resp = openai.ChatCompletion.create(
        model=mod,
        messages=mod_input,
        temperature=0 # degree of randomness 
    )
    return resp.choices[0].message["content"]

The above is not working as rate limits asssociated with my tier of key.

## Guidance on Prompts

Longer, more specific prompts are specified. You can use delimeters such as:

* `"""`
* ` ``` `
* `---`
* `<>`
* `<tag></tag>` (XML-flavoured tags)

Then use the available tags to stucture your prompt as follows:


In [30]:
war_n_peace  = """
Some scintillating text that you want to summarise  / analyse / do something with \
ensuring enouhg context is provided.
"""
# now for the instruction
usr_input = f"""Highlight any typos in the text delimited by triple hyphens\
    ---{war_n_peace}---"""
usr_input

'Highlight any typos in the text delimited by triple hyphens    ---\nSome scintillating text that you want to summarise  / analyse / do something with ensuring enouhg context is provided.\n---'